<a href="https://colab.research.google.com/github/hyunaeee/KU-SW-Academy/blob/main/1006.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Tokenizing
[Feature Extraction]
- sent_tokenize; 문장(구두점) => 문단 => 문서
- word_tokenize; regex; tweet; term/token/word/subword ...
                 => 용어 => 어휘집합(Voca)
                 => Text.vocab() = FreqDist = Count
[Featrue Selection]
=> Zipf; 자연어, 순위의 역과 빈도로 나열했을때 같더라, 전 순위보다 2배 가량 많이 나옴
       ; 고빈도(50%, 상위 소수의 단어) - 중빈도(*) - 저빈도(25%, 희소한)
         => TF-IDF, PRP->Okapi->BM25 => TF-IDF
=> Heaps; corpus가 커질수록 unique 단어들이 (특정 법칙) 증가
        ; k,b=>.4~.6 => feature 수를 예측 / Zipf 양질의 feature

[Korean]
- Morpheme/POS
  -> 형태소/형태소/형태소
  -> NN    VV   EX
- Grammar -> Rules => RNN
  -> NP      VP  => S
=> N-gram(N개의 gram=token) => LM
   1(uni), 2(bi), 3(tri), 4(...), ... (X)
𝑷(𝒙𝒊|𝒙𝒊− 𝒏−𝟏 ,...,𝒙𝒊−𝟏) => N-gram P(A,B) = ? A=배, B=고프다, 경우의 수를 모두 조합 => X P(A,B) => P(B|A)P(A) P(B|A) => P(A,B)/P(A) ------ ---- bigram uigram 배+?; MLE P(freq(A+B)/freq(A)) N N (단어,단어), (단어,단어) => LM P(C|A,B) => P(A,B,C)/P(A,B) => (단어,단어,단어) P(A,B,C,D,E,F,G,H,...,Z) => P(Z|A,...,Y)P(Y|A,...,X)... => 0 Markov Assumption(1st, 2nd, ..) P(A,B,C) => P(C|A,B)P(B|A)P(A) ~= P(B,C) P(A,B,C,D,E,F,G,H,...,Z) => P(Z|A,...,Y)P(Y|A,...,X)... ~= P(Z|Y)P(Y|X)P(X|..)... 1st Markov Assumption => Bigram 2nd Markov Assumption => Trigram P(A,B,C,D,E,F,G,H,...,Z) => P(Z|A,...,Y)P(Y|A,...,X)... ~= P(Z|XY)P(Y|WX)P(X|VW)... Z|XA, XB, XC, .., XY ABC ABD=0 ABE ACA ACG ACE=0 ... => 아 ___ => Sequence(L;given->R) => RNN;LSTM;GRU / Attention -----> <----- A B C [mask] E F G; LM? => Auto-encoder - - - - P(A) P(B|A) ... A B C D E F ...

def ngram(s, n=2):
    rst = list()
    for i in range(len(s)-(n-1)):
        rst.append(tuple(s[i:i+n]))
    return rst
from konlpy.corpus import kolaw
corpus = kolaw.open(kolaw.fileids()[0]).read()
from konlpy.tag import Komoran
ma = Komoran()
Python        JPype        Java(JVM)
Komoran() --------------> Komoran()
id        <--------------
from nltk.tokenize import word_tokenize
from nltk.text import Text
tokens = word_tokenize(corpus)
bigram = ngram(tokens)
biText = Text(bigram)
unigram = ngram(tokens, 1)
len(list(filter(lambda t:t[0] == '법률이', unigram)))
57
from nltk.probability import FreqDist

a = FreqDist(list(filter(lambda t:t[0] == '법률이', bigram)))
for k, v in a.items():
    print(f'{k}-{v} => P({k}|법률이)={v/sum(a.values())}')
('법률이', '정하는')-48 => P(('법률이', '정하는')|법률이)=0.8421052631578947
('법률이', '정한')-7 => P(('법률이', '정한')|법률이)=0.12280701754385964
('법률이', '확정된')-1 => P(('법률이', '확정된')|법률이)=0.017543859649122806
('법률이', '헌법에')-1 => P(('법률이', '헌법에')|법률이)=0.017543859649122806
# 네이버 뉴스 수집기 - Crawler
from requests import get
from bs4 import BeautifulSoup
import re

urls = list()
seens = list()

path = 'naver/'
urls.append('https://news.naver.com/')

headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

while urls:
    url = urls.pop(0)
    seens.append(url)

    dom = BeautifulSoup(get(url,headers=headers).text, 'html.parser')

    # 메뉴
    alist = dom.select('ul[role=menu] > li > a[href]')
    for a in alist:
        if re.search(r'sid1=\d{3}$', a.attrs['href']):
            if a.attrs['href'] not in urls and\
               a.attrs['href'] not in seens:
                urls.append(a.attrs['href'])

    # 기사링크 => 정교하게
    alist = dom.select('''
                        .sh_text > a[href],
                        .cluster_text > a[href],
                        dt > a[href]
                    ''')
    for a in alist:
        if a.attrs['href'] not in urls and\
           a.attrs['href'] not in seens:
            urls.append(a.attrs['href'])

    # 본문
    news = dom.select_one('#newsct_article')
    if news:
        file = re.search(r'(\d{10})', url).group(1)
        with open(f'{path}{file}.txt', 'w', encoding='utf-8') as fp:
            fp.write(news.text)
from os import listdir, mkdir
def fileids(path, ext='txt'):
    files = list(filter(lambda f:re.search(f'{ext}$', f),
                       listdir(path)))
    return list(map(lambda f:f'{path}/{f}', files))
corpus = list()
for f in fileids('naver'):
    with open(f, 'r', ecoding='utf8') as fp:
        corpus.append(fp.read())
from nltk.tokenize import sent_tokenize

wtText = list()
maText = list()
wtText.append(FreqDist())
maText.append(FreqDist())

for d in corpus:
    wtText.append(wtText[-1] + Text(word_tokenize(d)).vocab())

    rst = list()
    for s in sent_tokenize(re.sub(r'\s+', ' ', d)):
        rst.extend(ma.morphs(s))
    maText.append(maText[-1] + Text(rst).vocab())
wtText[-1].B(), wtText[-1].N(), \
maText[-1].B(), maText[-1].N()
(14867, 33114, 7014, 62327)
list(zip(wtText[-1].most_common(10), maText[-1].most_common(10)))
[(('.', 1572), ('하', 2556)),
 ((',', 732), ('.', 2136)),
 (('(', 390), ('이', 1867)),
 ((')', 390), ('는', 1408)),
 (("''", 231), ('다', 1384)),
 (('``', 203), ('ㄴ', 1365)),
 (('있다', 192), ('을', 1303)),
 (("'", 176), ('에', 1058)),
 (('“', 167), ('의', 880)),
 (('”', 165), ('를', 806))]
import matplotlib.pyplot as plt

N = 100
X1 = list(map(lambda r:1/(r+1), range(N)))
X2 = list(map(lambda r:
              wtText[-1].get(r)/wtText[-1].get(wtText[-1].max()),
         sorted(wtText[-1], key=wtText[-1].get, reverse=True)[:N]))
X3 = list(map(lambda r:
              maText[-1].get(r)/maText[-1].get(maText[-1].max()),
         sorted(maText[-1], key=maText[-1].get, reverse=True)[:N]))

plt.plot(range(N), X1)
plt.plot(range(N), X2)
plt.plot(range(N), X3)
[<matplotlib.lines.Line2D at 0x2a3377ca0>]

sum(list(map(lambda r:
              maText[-1].freq(r),
         sorted(maText[-1], key=maText[-1].get, reverse=True)[:60])))
0.4746418085259999
maText[-1].most_common(100)[60:]
[('말', 95),
 ('대', 95),
 ('ㅂ니다', 94),
 ('한', 93),
 ('시', 93),
 ('3', 89),
 ('위하', 89),
 ('그', 88),
 ('주', 84),
 ('지만', 84),
 ('없', 82),
 ('보', 81),
 ('한국', 81),
 ('6', 79),
 ('다는', 77),
 ('겠', 76),
 ('A', 75),
 ('억', 72),
 ('중', 72),
 ('일본', 72),
 ('부터', 70),
 ('라고', 68),
 ('기술', 68),
 ('지나', 67),
 ('서울', 67),
 ('대통령', 66),
 ('?', 66),
 ('밝히', 66),
 ('[', 66),
 ('=', 66),
 (']', 66),
 ('▲', 66),
 ('통하', 65),
 ('아서', 65),
 ('크', 65),
 ('성', 65),
 ('시장', 65),
 ('원', 64),
 ('차', 63),
 ('던', 62)]
k = 22 #10-100
b = .52
plt.plot(range(len(maText))[1:],
         [t.B() for t in maText][1:])
plt.plot(range(len(maText))[1:],
         list(map(lambda n:k*n.N()**b, maText))[1:])
[<matplotlib.lines.Line2D at 0x2a7298610>]

k*maText[-1].N()**b, maText[-1].B()
(6849.4281921751635, 7014)
sum(list(map(lambda r:
              maText[-1].freq(r),
         sorted(maText[-1], key=maText[-1].get)[:4600])))
0.10180178734737322
gram1 = list()
gram2 = list()
gram3 = list()

for d in corpus:
    rst = list()
    for s in sent_tokenize(re.sub(r'\s+', ' ', d)):
        rst.extend(ma.morphs(s))
    gram1 += ngram(rst, 1)
    gram2 += ngram(rst, 2)
    gram3 += ngram(rst, 3)
len(gram1), len(gram2), len(gram3)
(62327, 62240, 62153)
from collections import Counter
gram1_cnt = Counter(gram1)
gram2_cnt = Counter(gram2)
gram3_cnt = Counter(gram3)
P(단어?|시작단어) => P(시작단어,단어?)/P(시작단어)
               => freq(bigram(시작단어,단어?))/freq(unigram(시작단어))
seed = '대통령'
gram1_cnt.get((seed,))

list(map(lambda k:{k:gram2_cnt.get(k),
                   'prob':gram2_cnt.get(k)/gram1_cnt.get((seed,))},
         list(filter(lambda k:k[0] == seed, gram2_cnt))))
[{('대통령', '을'): 5, 'prob': 0.07575757575757576},
 {('대통령', '에'): 1, 'prob': 0.015151515151515152},
 {('대통령', '과'): 3, 'prob': 0.045454545454545456},
 {('대통령', '후보'): 1, 'prob': 0.015151515151515152},
 {('대통령', '이'): 10, 'prob': 0.15151515151515152},
 {('대통령', '실'): 6, 'prob': 0.09090909090909091},
 {('대통령', '비서실장'): 1, 'prob': 0.015151515151515152},
 {('대통령', '에게'): 1, 'prob': 0.015151515151515152},
 {('대통령', '은'): 25, 'prob': 0.3787878787878788},
 {('대통령', '의'): 6, 'prob': 0.09090909090909091},
 {('대통령', '부부'): 1, 'prob': 0.015151515151515152},
 {('대통령', '사저'): 1, 'prob': 0.015151515151515152},
 {('대통령', '실은'): 1, 'prob': 0.015151515151515152},
 {('대통령', '님'): 1, 'prob': 0.015151515151515152},
 {('대통령', ','): 1, 'prob': 0.015151515151515152},
 {('대통령', '표창'): 1, 'prob': 0.015151515151515152},
 {('대통령', '발언'): 1, 'prob': 0.015151515151515152}]
seed = '“'
gram1_cnt.get((seed,))

list(map(lambda k:{k:gram2_cnt.get(k),
                   'prob':gram2_cnt.get(k)/gram1_cnt.get((seed,))},
         list(filter(lambda k:k[0] == seed, gram2_cnt))))
[{('“', '전'): 1, 'prob': 0.006060606060606061},
 {('“', '어차피'): 1, 'prob': 0.006060606060606061},
 {('“', '제'): 3, 'prob': 0.01818181818181818},
 {('“', '사후'): 1, 'prob': 0.006060606060606061},
 {('“', '저'): 4, 'prob': 0.024242424242424242},
 {('“', '지금'): 2, 'prob': 0.012121212121212121},
 {('“', '날'): 1, 'prob': 0.006060606060606061},
 {('“', '곧'): 1, 'prob': 0.006060606060606061},
 {('“', '이번'): 2, 'prob': 0.012121212121212121},
 {('“', '현재'): 3, 'prob': 0.01818181818181818},
 {('“', '후회하지 않아'): 1, 'prob': 0.006060606060606061},
 {('“', '어떤'): 1, 'prob': 0.006060606060606061},
 {('“', '태어나'): 1, 'prob': 0.006060606060606061},
 {('“', '제가'): 2, 'prob': 0.012121212121212121},
 {('“', '영화'): 1, 'prob': 0.006060606060606061},
 {('“', '1980년대'): 1, 'prob': 0.006060606060606061},
 {('“', '홍콩'): 1, 'prob': 0.006060606060606061},
 {('“', '그래서'): 1, 'prob': 0.006060606060606061},
 {('“', '저도'): 1, 'prob': 0.006060606060606061},
 {('“', '이'): 1, 'prob': 0.006060606060606061},
 {('“', '최신'): 1, 'prob': 0.006060606060606061},
 {('“', '('): 3, 'prob': 0.01818181818181818},
 {('“', '본인'): 1, 'prob': 0.006060606060606061},
 {('“', '게이츠'): 1, 'prob': 0.006060606060606061},
 {('“', '매카시'): 1, 'prob': 0.006060606060606061},
 {('“', '세계'): 1, 'prob': 0.006060606060606061},
 {('“', '새롭'): 1, 'prob': 0.006060606060606061},
 {('“', '하와이'): 1, 'prob': 0.006060606060606061},
 {('“', '이역만리'): 1, 'prob': 0.006060606060606061},
 {('“', '미국'): 1, 'prob': 0.006060606060606061},
 {('“', '드라마'): 1, 'prob': 0.006060606060606061},
 {('“', '한국'): 3, 'prob': 0.01818181818181818},
 {('“', '자이'): 1, 'prob': 0.006060606060606061},
 {('“', '세종대왕'): 1, 'prob': 0.006060606060606061},
 {('“', '반려'): 1, 'prob': 0.006060606060606061},
 {('“', '앞으로'): 2, 'prob': 0.012121212121212121},
 {('“', '영상'): 1, 'prob': 0.006060606060606061},
 {('“', '구조'): 1, 'prob': 0.006060606060606061},
 {('“', '계단'): 1, 'prob': 0.006060606060606061},
 {('“', '기존'): 1, 'prob': 0.006060606060606061},
 {('“', '향후'): 1, 'prob': 0.006060606060606061},
 {('“', '거래'): 1, 'prob': 0.006060606060606061},
 {('“', '연기자'): 1, 'prob': 0.006060606060606061},
 {('“', '쉬'): 1, 'prob': 0.006060606060606061},
 {('“', '두'): 1, 'prob': 0.006060606060606061},
 {('“', '26년'): 1, 'prob': 0.006060606060606061},
 {('“', '빙빙'): 2, 'prob': 0.012121212121212121},
 {('“', '주영'): 2, 'prob': 0.012121212121212121},
 {('“', '사랑해'): 1, 'prob': 0.006060606060606061},
 {('“', '중국'): 1, 'prob': 0.006060606060606061},
 {('“', '보이'): 1, 'prob': 0.006060606060606061},
 {('“', '코로나'): 2, 'prob': 0.012121212121212121},
 {('“', '여성'): 1, 'prob': 0.006060606060606061},
 {('“', '어쩌면'): 1, 'prob': 0.006060606060606061},
 {('“', '합작'): 1, 'prob': 0.006060606060606061},
 {('“', '지난해'): 1, 'prob': 0.006060606060606061},
 {('“', '‘'): 1, 'prob': 0.006060606060606061},
 {('“', '디지털'): 1, 'prob': 0.006060606060606061},
 {('“', '윤'): 1, 'prob': 0.006060606060606061},
 {('“', '도요타'): 1, 'prob': 0.006060606060606061},
 {('“', '장기'): 1, 'prob': 0.006060606060606061},
 {('“', 'OLED'): 1, 'prob': 0.006060606060606061},
 {('“', '가장'): 1, 'prob': 0.006060606060606061},
 {('“', '폴'): 1, 'prob': 0.006060606060606061},
 {('“', '2024년'): 1, 'prob': 0.006060606060606061},
 {('“', 'UDC'): 2, 'prob': 0.012121212121212121},
 {('“', 'B'): 1, 'prob': 0.006060606060606061},
 {('“', '설계'): 1, 'prob': 0.006060606060606061},
 {('“', '김건희'): 3, 'prob': 0.01818181818181818},
 {('“', '그'): 3, 'prob': 0.01818181818181818},
 {('“', '오늘'): 1, 'prob': 0.006060606060606061},
 {('“', '연간'): 1, 'prob': 0.006060606060606061},
 {('“', '우리'): 1, 'prob': 0.006060606060606061},
 {('“', '리모컨'): 1, 'prob': 0.006060606060606061},
 {('“', '궁극'): 1, 'prob': 0.006060606060606061},
 {('“', '스타트'): 1, 'prob': 0.006060606060606061},
 {('“', '한국의 전통'): 1, 'prob': 0.006060606060606061},
 {('“', '협동'): 1, 'prob': 0.006060606060606061},
 {('“', '지나'): 3, 'prob': 0.01818181818181818},
 {('“', '빠르'): 1, 'prob': 0.006060606060606061},
 {('“', '국채'): 2, 'prob': 0.012121212121212121},
 {('“', '여전히'): 1, 'prob': 0.006060606060606061},
 {('“', '보다'): 1, 'prob': 0.006060606060606061},
 {('“', '계속'): 1, 'prob': 0.006060606060606061},
 {('“', '엔'): 1, 'prob': 0.006060606060606061},
 {('“', '경각심'): 1, 'prob': 0.006060606060606061},
 {('“', '국제'): 1, 'prob': 0.006060606060606061},
 {('“', '상당히'): 1, 'prob': 0.006060606060606061},
 {('“', '가계부채'): 1, 'prob': 0.006060606060606061},
 {('“', '올해'): 2, 'prob': 0.012121212121212121},
 {('“', '고금리'): 1, 'prob': 0.006060606060606061},
 {('“', '내년'): 2, 'prob': 0.012121212121212121},
 {('“', '생성'): 1, 'prob': 0.006060606060606061},
 {('“', 'AI'): 1, 'prob': 0.006060606060606061},
 {('“', '소유'): 1, 'prob': 0.006060606060606061},
 {('“', '예전'): 2, 'prob': 0.012121212121212121},
 {('“', '아마존'): 1, 'prob': 0.006060606060606061},
 {('“', '분초'): 1, 'prob': 0.006060606060606061},
 {('“', '초'): 1, 'prob': 0.006060606060606061},
 {('“', '고령자'): 1, 'prob': 0.006060606060606061},
 {('“', '돌'): 1, 'prob': 0.006060606060606061},
 {('“', '범죄자'): 1, 'prob': 0.006060606060606061},
 {('“', '가'): 1, 'prob': 0.006060606060606061},
 {('“', '도저히'): 1, 'prob': 0.006060606060606061},
 {('“', '사퇴'): 1, 'prob': 0.006060606060606061},
 {('“', '위원장'): 2, 'prob': 0.012121212121212121},
 {('“', '나가'): 1, 'prob': 0.006060606060606061},
 {('“', '후보자'): 1, 'prob': 0.006060606060606061},
 {('“', '김'): 2, 'prob': 0.012121212121212121},
 {('“', '라고'): 1, 'prob': 0.006060606060606061},
 {('“', '코레일유통'): 1, 'prob': 0.006060606060606061},
 {('“', '선도'): 1, 'prob': 0.006060606060606061},
 {('“', '학생'): 1, 'prob': 0.006060606060606061},
 {('“', '대학'): 1, 'prob': 0.006060606060606061},
 {('“', '처음'): 1, 'prob': 0.006060606060606061},
 {('“', 'R'): 1, 'prob': 0.006060606060606061},
 {('“', '무슨'): 1, 'prob': 0.006060606060606061},
 {('“', '재정'): 1, 'prob': 0.006060606060606061},
 {('“', '국회'): 1, 'prob': 0.006060606060606061},
 {('“', '그동안'): 1, 'prob': 0.006060606060606061},
 {('“', '교육'): 1, 'prob': 0.006060606060606061},
 {('“', '긴축'): 1, 'prob': 0.006060606060606061},
 {('“', '백서'): 1, 'prob': 0.006060606060606061},
 {('“', '블랙리스트'): 2, 'prob': 0.012121212121212121},
 {('“', '모두'): 1, 'prob': 0.006060606060606061},
 {('“', '민주당'): 1, 'prob': 0.006060606060606061},
 {('“', '그런'): 1, 'prob': 0.006060606060606061},
 {('“', '여기'): 1, 'prob': 0.006060606060606061},
 {('“', 'OO'): 1, 'prob': 0.006060606060606061},
 {('“', '원색'): 1, 'prob': 0.006060606060606061},
 {('“', '대법원장'): 2, 'prob': 0.012121212121212121},
 {('“', '부디'): 1, 'prob': 0.006060606060606061}]
# key로 시작하는 freq 찾는 함수
def find(key, gram):
    n = len(key)
    return [(k,gram.get(k)) for k in list(gram.keys())
            if k[:n] == key]
# P(ABC) => P(C|AB)
#        => P(C|A?)\
seed = ('대통령',)
for i in range(20):
    n = len(seed)
    if n==1:
        p1=gram1_cnt
        p2=gram2_cnt
    elif n==2:
        p1=gram2_cnt
        p2=gram3_cnt

    given = find(seed, p1)[0][1]
    candidates = list(map(lambda r:(r[0], r[1]/given),
                          find(seed, p2)))
    best = list(sorted(candidates, key=lambda r:r[1],
                       reverse=True))[0]
    print(best)
    seed = best[0][-2:]
(('대통령', '은'), 0.3787878787878788)
(('대통령', '은', '"'), 0.2)
(('은', '"', '외국'), 0.043478260869565216)
(('"', '외국', '계'), 1.0)
(('외국', '계', '공룡'), 0.4)
(('계', '공룡', '플랫폼'), 1.0)
(('공룡', '플랫폼', '실태'), 0.5)
(('플랫폼', '실태', '파악'), 1.0)
(('실태', '파악', '…'), 0.5)
(('파악', '…', '조세'), 1.0)
(('…', '조세', '정'), 1.0)
(('조세', '정', '의'), 1.0)
(('정', '의', '실현'), 1.0)
(('의', '실현', '시급'), 0.3333333333333333)
(('실현', '시급', '"'), 1.0)
(('시급', '"', '2004'), 1.0)
(('"', '2004', '년'), 1.0)
(('2004', '년', '설립'), 0.5)
(('년', '설립', '되'), 1.0)
(('설립', '되', 'ㄴ'), 1.0)
# 공백이 포함된 ngram
ngram1 = Counter(ngram(' '.join(corpus), 1))
ngram2 = Counter(ngram(' '.join(corpus), 2))
ngram3 = Counter(ngram(' '.join(corpus), 3))
ngram4 = Counter(ngram(' '.join(corpus), 4))
# 자동 띄어쓰기
'임명 동의안이 상당 기간 표류한 데다 과반 의석을 가진 더불어민주당이 부결 카드를 만지작거리면서 정치권이 당리당략에 따라 사법부 수장을 볼모로 잡아선 안 된다는 지적이다.'.replace(' ', '')
'임명동의안이상당기간표류한데다과반의석을가진더불어민주당이부결카드를만지작거리면서정치권이당리당략에따라사법부수장을볼모로잡아선안된다는지적이다.'
find(('임',), ngram1)
[(('임',), 74)]
list(sorted(find(tuple('동의'), ngram3),
            key=lambda r:r[1], reverse=True))[0]
(('동', '의', ' '), 4)
Ngram => LanguageModel
빈도 => MLE => Conditional Prob. (과정)자체가 NLU
Ngrams => 다음 단어는? 최대확률추정 NLG
=> Tokenizing
image.png

P(쎈|알잘딱깔) => 1
P(?|알잘딱깔) => 1/0.~ => > 1,
P(쎈|알잘딱깔) => P(알잘딱깔쎈)/P(알잘딱깔)
P(잘|알)*P(딱|알잘)*P(깔|알잘딱)*P(쎈|알잘딱깔)
P(알잘)   P(알잘딱)   P(알잘딱깔)       P(전체조합)
------ X ------- X -------    => ----------
P(알)     P(알잘)   P (알잘딱)        P(첫글자)
=> Cohesion Score => 분모 = 1
find(tuple('레'), ngram1)
[(('레',), 130)]
find(tuple('레이'), ngram2)
[(('레', '이'), 81)]
find(tuple('레이'), ngram3)
[(('레', '이', ' '), 46),
 (('레', '이', "'"), 1),
 (('레', '이', '의'), 9),
 (('레', '이', '라'), 1),
 (('레', '이', '는'), 6),
 (('레', '이', '크'), 1),
 (('레', '이', '션'), 3),
 (('레', '이', '를'), 3),
 (('레', '이', '가'), 1),
 (('레', '이', '저'), 1),
 (('레', '이', '에'), 2),
 (('레', '이', ','), 1),
 (('레', '이', '스'), 1),
 (('레', '이', '산'), 1),
 (('레', '이', '.'), 1),
 (('레', '이', '['), 1),
 (('레', '이', '학'), 1),
 (('레', '이', '시'), 1)]
81/130, (46/81)*(1/81)*(9/81)
(0.6230769230769231, 0.0007790140391877931)
[어근/어간 형태소]+형태소(형식/의존)
*     |
  ****|
      |***
대 통 령  이
        께서..